In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cuda


In [4]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [5]:
#Dataloader

#Path for training and testing directory
train_path='E:/object-detect/Pytorch_Tutorials-master=CNN/Pytorch_Tutorials-master/archive (67)/seg_train/'
test_path='E:/object-detect/Pytorch_Tutorials-master=CNN/Pytorch_Tutorials-master/archive (67)/seg_test/'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [6]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [7]:
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [8]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        


In [9]:
model=ConvNet(num_classes=6).to(device)

In [10]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [11]:
num_epochs=10

In [12]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [13]:
print(train_count,test_count)

14034 3000


In [15]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model-1')
        best_accuracy=test_accuracy
    
       


Epoch: 0 Train Loss: tensor(0.1410) Train Accuracy: 0.9603819295995439 Test Accuracy: 0.7263333333333334
Epoch: 1 Train Loss: tensor(0.1401) Train Accuracy: 0.9601681630326351 Test Accuracy: 0.7446666666666667
Epoch: 2 Train Loss: tensor(0.1358) Train Accuracy: 0.9630183839247541 Test Accuracy: 0.7533333333333333
Epoch: 3 Train Loss: tensor(0.0921) Train Accuracy: 0.9744905230155337 Test Accuracy: 0.7213333333333334
Epoch: 4 Train Loss: tensor(0.0818) Train Accuracy: 0.9764144221177141 Test Accuracy: 0.7593333333333333
Epoch: 5 Train Loss: tensor(0.0815) Train Accuracy: 0.9796209206213482 Test Accuracy: 0.7063333333333334
Epoch: 6 Train Loss: tensor(0.0938) Train Accuracy: 0.9753455892831695 Test Accuracy: 0.7666666666666667
Epoch: 7 Train Loss: tensor(0.0814) Train Accuracy: 0.9786233433091065 Test Accuracy: 0.7293333333333333
Epoch: 8 Train Loss: tensor(0.0903) Train Accuracy: 0.9760581445061992 Test Accuracy: 0.76
Epoch: 9 Train Loss: tensor(0.1061) Train Accuracy: 0.971854068690323

In [16]:
import torch
from torch.autograd import Variable

# Model training and saving the best model
best_accuracy = 0.0

# Assuming train_loader and test_loader are already defined, along with the model, optimizer, and loss_function
for epoch in range(num_epochs):
    
    # Evaluation and training on the training dataset
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)
        
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    
    # Evaluation on the testing dataset
    model.eval()
    test_accuracy = 0.0
    
    # Ensure that test_loader is properly loaded and contains data
    if len(test_loader.dataset) == 0:
        raise ValueError("Test dataset is empty. Please check your data loading process.")
    
    test_count = len(test_loader.dataset)
    
    for i, (images, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
    
    # Avoid division by zero, in case test_count is zero
    if test_count > 0:
        test_accuracy = test_accuracy / test_count
    else:
        test_accuracy = 0.0
        print("Warning: Test dataset is empty. Test accuracy cannot be computed.")
    
    print(f'Epoch: {epoch} Train Loss: {train_loss} Train Accuracy: {train_accuracy} Test Accuracy: {test_accuracy}')
    
    # Save the best model
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy = test_accuracy


Epoch: 0 Train Loss: 2.3647212982177734 Train Accuracy: 0.6966652415562206 Test Accuracy: 0.13466666666666666
Epoch: 1 Train Loss: 1.4023983478546143 Train Accuracy: 0.7672794641584723 Test Accuracy: 0.19733333333333333
Epoch: 2 Train Loss: 0.7794827222824097 Train Accuracy: 0.8202223172295853 Test Accuracy: 0.243
Epoch: 3 Train Loss: 0.4195474088191986 Train Accuracy: 0.8821433661108736 Test Accuracy: 0.27466666666666667
Epoch: 4 Train Loss: 0.28904178738594055 Train Accuracy: 0.9177711272623629 Test Accuracy: 0.154
Epoch: 5 Train Loss: 0.19051708281040192 Train Accuracy: 0.9419267493230725 Test Accuracy: 0.09866666666666667
Epoch: 6 Train Loss: 0.16519196331501007 Train Accuracy: 0.9492660681202794 Test Accuracy: 0.229
Epoch: 7 Train Loss: 0.123848557472229 Train Accuracy: 0.9641584722816018 Test Accuracy: 0.11233333333333333
Epoch: 8 Train Loss: 0.09807977080345154 Train Accuracy: 0.9717115576457176 Test Accuracy: 0.16
Epoch: 9 Train Loss: 0.09423483163118362 Train Accuracy: 0.97449